<a href="https://colab.research.google.com/github/geraldmaale/dp-deepesn/blob/main/dp_opacus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.autonotebook import tqdm

from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, Normalize, ToTensor
from torch.utils.data import DataLoader

In [3]:
if torch.cuda.is_available():
    print(f'Torch will use GPU ... {torch.cuda.get_device_name()}')
else:
    print('Torch will use CPU ...')


Torch will use GPU ... Tesla T4


In [4]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

BATCH_SIZE = 192 

train_ds = CIFAR10('.',
                    train=True,
                    download=True,
                    transform=Compose([ToTensor(), Normalize(IMAGENET_MEAN, IMAGENET_STD)]))

train_loader = DataLoader(train_ds,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=8,
                            pin_memory=True,
                            drop_last=True)


Extracting ./cifar-10-python.tar.gz to .


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
model = Net().train().to('cuda')
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
criterion = nn.CrossEntropyLoss()

In [15]:
model

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [23]:
# for epoch in range(2):  # loop over the dataset multiple times

running_loss = 0.0
# for i, data in enumerate(train_loader, 0):
for x, y in tqdm(train_loader):
    # get the inputs; data is a list of [inputs, labels]
    # inputs, labels = data
    x = x.to('cuda')
    y = y.to('cuda')

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    logits = model(x)
    loss = criterion(logits, y)
    loss.backward()
    optimizer.step()

        # print statistics
        # running_loss += loss.item()
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Finished Training


## Differential Privacy

In [24]:
!pip install opacus

     |████████████████████████████████| 114 kB 8.3 MB/s 


In [28]:
import opacus

In [34]:
privacy_engine = opacus.PrivacyEngine(
    model,
    batch_size=BATCH_SIZE,
    sample_rate=0.1,
    alphas=range(2,20),
    noise_multiplier=1.3, 
    max_grad_norm=1.0,
)

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:646: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:230: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


In [35]:
privacy_engine.get_privacy_spent()

(0.5498811997740483, 19.0)

In [36]:
privacy_engine.attach(optimizer=optimizer)

In [38]:
# for epoch in range(2):  # loop over the dataset multiple times

running_loss = 0.0
# for i, data in enumerate(train_loader, 0):
for x, y in tqdm(train_loader):
    # get the inputs; data is a list of [inputs, labels]
    # inputs, labels = data
    x = x.to('cuda')
    y = y.to('cuda')

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    logits = model(x)
    loss = criterion(logits, y)
    loss.backward()
    optimizer.step()

        # print statistics
        # running_loss += loss.item()
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



Finished Training


In [39]:
privacy_engine.get_privacy_spent()

(8.925875788742273, 4.0)